Solver

In [1]:
using JuMP
using Gurobi
using CSV
using DataFrames

Data Importation

In [2]:
budgets_demand = CSV.File("./data/budgets_demand.csv",header=0) |> Tables.matrix;
impression_data = CSV.File("./data/impression_per_age.csv",header=0) |> Tables.matrix; # plaform, age group, impressions rate
company_rcs_data = CSV.File("./data/company_rcs.csv",header=0) |> Tables.matrix; # company, rps
platform_variables_data = CSV.File("./data/platform_variables.csv",header=0) |> Tables.matrix; # platform, variable, value


Clean Data

In [3]:
# define dictionaries to use

# B: budget
budget_dict = Dict() # dict of (company) -> budget; 50 companies
for i in 2:size(budgets_demand,1)
    budget_dict[budgets_demand[i, 1]] = parse(Int, budgets_demand[i, 2])
end

# R_a: total reach demanded for each age group
demand_dict = Dict() # dict of (company, age group) -> demand; 50 companies, 4 age groups
age_groups = ["18-24", "25-34", "35-44", "45-60"]
for i in 2:size(budgets_demand,1)
    for j in 3:size(budgets_demand,2)
        demand_dict[(budgets_demand[i, 1], budgets_demand[1,j])] = parse(Int, budgets_demand[i,j])
    end
end

# impression rate per age group (a_ap*i_p)
impressions_per_age = Dict() # dict of (platform, age group) -> impression rate
for i in 2:size(impression_data,1)
    str_value = string(impression_data[i, 3])  # Convert to regular String
    impressions_per_age[(impression_data[i, 1], impression_data[i, 2])] = parse(Float64, str_value)  # Parse to Int
end


# company revenue per conversion
company_rcs = Dict() # dict of company -> r_c
for i in 1:size(company_rcs_data,2)
    # row 1 is company name, row 2 is r_c
    company_rcs[company_rcs_data[1, i]] = parse(Float64, company_rcs_data[2, i])
end

# platform variables
# constant for all companies need for each platform, 
    # w_p: click_rate
    # i_p: impression_rate
    # f_p: average_impression_rate
    # c_p: conversion rate
    # s_p: cost per click 

platform_variables = Dict() # dict of (platform, variable) -> value
for i in 2:size(platform_variables_data, 2)
    for j in 2:size(platform_variables_data, 1)
        platform_variables[platform_variables_data[1, i], platform_variables_data[j, 1]] = parse(Float64, platform_variables_data[j, i])
    end
end



In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


Solver

In [8]:
platforms_l = ["Facebook", "Instagram", "Twitter"] # keept this order
companies_l = collect(keys(budget_dict)) # keept this order
age_groups = ["18-24", "25-34", "35-44", "45-60"]


# model for each company
optimized_solutions= Dict() # dict of company -> {t -> [t_p], optimal_value -> value}

for i in 1:length(companies_l)
    company = companies_l[i]
    println("Optimizing for company: ", company)
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))

    ### decision variable ###
    # hours of advertisement allocated to platform p by company c: t_p
    @variable(model, t[1:length(platforms_l)] >= 0);

    ### constraints ###
    # reach for certain age group must reach a min
    @constraint(model, [a in 1:length(age_groups)], sum(((impressions_per_age[platforms_l[p], age_groups[a]] * t[p])/platform_variables[platforms_l[p], "f_p"] for p in 1:length(platforms_l))) >= demand_dict[company, age_groups[a]]);

    # budget constraint
    # @constraint(model, sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l)) <= budget_dict[company]);


    ### objective function ### 
    @objective(model, Min,
    sum((
        sum(platform_variables[platforms_l[p], "s_p"] * (platform_variables[platforms_l[p], "w_p"]*(platform_variables[platforms_l[p], "i_p"]*t[p])/platform_variables[platforms_l[p], "f_p"]) for p in 1:length(platforms_l)))
    ));

    optimize!(model)
    optimized_solutions[company] = Dict()

    try
        optimized_solutions[company]["optimal_budget"] = objective_value(model)
        optimized_solutions[company]["t"] = value.(t)
    catch e
        optimized_solutions[company]["optimal_budget"] = -1 # no solution
        optimized_solutions[company]["t"] = -1
    end
end



Optimizing for company: Silk & Style
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 4 rows, 3 columns and 12 nonzeros
Model fingerprint: 0x67932deb
Coefficient statistics:
  Matrix range     [6e-01, 3e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve time: 0.00s
Presolved: 4 rows, 3 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.147500e+03   0.000000e+00      0s
       2    1.3666207e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.366620743e+03

User-callback calls 42, time in user-callback 0.00 sec
Optimizing for company: Vogue Visions
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)


In [9]:
# optimized_solutions["Silk & Style"]

for (company, solution) in optimized_solutions
    println("Company: ", company)
    println("Optimal budget: ", solution["optimal_budget"])
    println("Optimal t: ", solution["t"])
    println("budget: ", budget_dict[company])
    println()
end

Company: Silk & Style
Optimal budget: 1366.6207425434357
Optimal t: [0.0, 57.23133245443417, 532.3694068256043]
budget: 1420

Company: Vogue Visions
Optimal budget: 1378.2910068892033
Optimal t: [0.0, 0.0, 603.6807129918278]
budget: 2668

Company: Hearth Harmony
Optimal budget: 1367.9654333968488
Optimal t: [0.0, 517.9866195764649, 0.0]
budget: 1169

Company: Silicon Saga
Optimal budget: 1853.9465729011915
Optimal t: [0.0, 442.65064352741, 299.9975193002428]
budget: 2388

Company: Furnish Fine
Optimal budget: 1511.6731340412246
Optimal t: [0.0, 7.519025221838587, 653.4037435297679]
budget: 851

Company: Gastronomy Guild
Optimal budget: 2500.581975026498
Optimal t: [0.0, 946.8572615913876, 0.0]
budget: 2808

Company: Dwell Delight
Optimal budget: 2232.8726106413083
Optimal t: [0.0, 845.4878371151332, 0.0]
budget: 652

Company: Cozy Corners
Optimal budget: 1292.2374580719793
Optimal t: [0.0, 0.0, 565.9899296624733]
budget: 2993

Company: Fitness Front
Optimal budget: 2374.0819457016282
O